# Research borrower reliability

## Data review

In [1]:
# download library
import pandas as pd

In [2]:
# read data
path = ''
data = pd.read_csv(path)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# look at first rows
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


- `days_employed` has negative values
- `days_employed` and `total_income` have NA values
- `eduacation` have similar words 'Среднее' and 'среднее'

### Filling in gaps

In [4]:
# check for missing values

# looking for missing values in `days_employed`
days_employed_nan = len(data[data['days_employed'].isna()]['days_employed'])
days_employed_nan_rate = days_employed_nan / len(data['days_employed'])

# lookin for missing values in `total_income`
total_income_nan = len(data[data['total_income'].isna()]['total_income'])
total_income_nan_rate = total_income_nan / len(data['total_income'])

print(f'Proportion of missing values in "days_employed": {days_employed_nan_rate:.0%}')
print(f'Proportion of missing values in "total_income": {total_income_nan_rate:.0%}')

Proportion of missing values in "days_employed": 10%
Proportion of missing values in "total_income": 10%


Fiiling in the missing values with the median

In [5]:
# median for `days_employed` and `total_income`
days_employed_median = data['days_employed'].median()
total_income_median = data['total_income'].median()

# filling in missing values
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
data['total_income'] = data['total_income'].fillna(total_income_median)

# check
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Check data for wrongs

Column `days_employed` have negative values. Conver it to absolute value of number

In [6]:
# apply abs function
data['days_employed'] = abs(data['days_employed'])
# check median
data['days_employed'].median()

1808.0534339280623

In [7]:
# `gender` have unknown value
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [8]:
# find row with this value
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [9]:
# drop and check
data = data.drop(10701)
data['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

In [10]:
# column `children` have negative values
data['children'].value_counts()

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [11]:
# convert to absolute value of number
data['children'] = abs(data['children'])
# check
data['children'].value_counts()

0     14148
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [12]:
# also column `children` have too large values (20). delete it. this doesn't affcet to reseach
data = data.drop(data[data['children'] == 20].index)
# check
data['children'].value_counts()

0    14148
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

### Change data type

In [13]:
# change type to int for `total_income`
data['total_income'] = data['total_income'].astype(int)
# check
data['total_income'].head()

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int32

### Delete duplicates

In [14]:
# delete duplicates in all table
data = data.drop_duplicates()

In [15]:
# find implicit duplicates
data['education'].value_counts()

среднее                13653
высшее                  4698
СРЕДНЕЕ                  770
Среднее                  705
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [16]:
# owerwrite values to lowercase
data['education'] = data['education'].str.lower()
# check
data['education'].value_counts()

среднее                15128
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

### Create additional dataframes

In [17]:
# new dfs with education and family data
data_education = data.loc[:,['education', 'education_id']]
data_family = data.loc[:, ['family_status_id', 'family_status']]

In [18]:
# delete duplicates
data_education = data_education.drop_duplicates()
data_family = data_family.drop_duplicates()
data_family = data_family.reset_index(drop=True)
data_education = data_education.reset_index(drop=True)

In [19]:
# delete columns
data.drop(columns=['education', 'family_status'], inplace=True)
#check
data.columns

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

### Income categorization

In [20]:
# create function

def income_category(income):
    """
    Возращает категорию дохода:
    - 0–30000 — 'E'
    - 30001–50000 — 'D'
    - 50001–200000 — 'C'
    - 200001–1000000 — 'B'
    1000001 и выше — 'A'
    """
    
    if income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    if income >= 1000001:
        return 'A'
    
# apply function
data['total_income_category'] = data['total_income'].apply(income_category)
# check
data.loc[:9, ['total_income', 'total_income_category']]

,total_income,total_income_category
0,253875,B
1,112080,C
2,145885,C
3,267628,B
4,158616,C
5,255763,B
6,240525,B
7,135823,C
8,95856,C
9,144425,C


### Purpose categorization

In [21]:
# create function

def purpose_group(purpose):
    """
    Возвращает категорию цели кредита
    """
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'недвижим' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if 'образов' in purpose:
        return 'получение образования'
        
# apply function
data['purpose_category'] = data['purpose'].apply(purpose_group)
# check
data.loc[:14, ['purpose', 'purpose_category']]

,purpose,purpose_category
0,покупка жилья,операции с недвижимостью
1,приобретение автомобиля,операции с автомобилем
2,покупка жилья,операции с недвижимостью
3,дополнительное образование,получение образования
4,сыграть свадьбу,проведение свадьбы
5,покупка жилья,операции с недвижимостью
6,операции с жильем,операции с недвижимостью
7,образование,получение образования
8,на проведение свадьбы,проведение свадьбы
9,покупка жилья для семьи,операции с недвижимостью


### Answer for questions

##### Is there relationship between number of children and loan repayment on time?

In [22]:
# check
data_children_debt = data.pivot_table(index='children', columns='debt', values='gender', aggfunc='count')
data_children_debt['rate_debt_children'] = data_children_debt[1] / data_children_debt[0]
data_children_debt

debt,0,1,rate_debt_children
children,,,
0,13043.0,1063.0,0.081500
1,4411.0,445.0,0.100884
2,1858.0,194.0,0.104413
3,303.0,27.0,0.089109
4,37.0,4.0,0.108108
5,9.0,NaN,NaN


Proportions clients without children near 8%. With children 9% - 10%

##### Is there relationship between family status and loan rapayment on time?

In [23]:
# merge table
data_family_debt = data.merge(data_family, on='family_status_id', how='left')
data_family_debt = data_family_debt[['family_status', 'debt', 'family_status_id']]
# pivot table
data_family_debt = data_family_debt.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
data_family_debt['rate_debt_family'] = data_family_debt[1] / data_family_debt[0]
data_family_debt

debt,0,1,rate_debt_family
family_status,,,
Не женат / не замужем,2528,273,0.107991
в разводе,1109,84,0.075744
вдовец / вдова,892,63,0.070628
гражданский брак,3765,385,0.102258
женат / замужем,11367,928,0.081640


Big proportion of debts from clients in a civil marriage and who are not married or not married: near 10%. 

##### Is there relationship between income and lone rapayment on time?

In [24]:
data_debt_incom_category = data.pivot_table(index='total_income_category', columns='debt', values='total_income', aggfunc='count')
data_debt_incom_category['rate_debt_income_category'] = data_debt_incom_category[1] / data_debt_incom_category[0]
data_debt_incom_category

debt,0,1,rate_debt_income_category
total_income_category,,,
A,23,2,0.086957
B,4666,354,0.075868
C,14623,1354,0.092594
D,329,21,0.063830
E,20,2,0.100000


Propotion of non-payers from 6% to 10%

##### How do different purposes affect to loan repaymnet on time?

In [25]:
data_debt_purpose_category = data.pivot_table(index='purpose_category', columns='debt', values='purpose', aggfunc='count')
data_debt_purpose_category['rate_debt_purpose'] = data_debt_purpose_category[1] / data_debt_purpose_category[0]
data_debt_purpose_category

debt,0,1,rate_debt_purpose
purpose_category,,,
операции с автомобилем,3891,401,0.103058
операции с недвижимостью,9997,780,0.078023
получение образования,3630,369,0.101653
проведение свадьбы,2143,183,0.085394


The largest share of debts is ralated to operations with cars and education: ~10%

## Results
- Clients with children are experiencing an increase in cases of loan arrears
- Clients with income before 30000 and between 50000 and 200000 have the most debt
- Most of the non-payers are clients whose purpose of the loan is related to a car and education